In [46]:
import numpy as np
import pandas as pd
import csv
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report


In [47]:
#stores the text into a dictionary where each TfidfVector can be found with the original message

readPath = 'TrainingData.csv'
data = pd.read_csv(readPath)

wholeText = data['Message1'].astype(str).tolist() + data['Message2'].astype(str).tolist()
wholeText = pd.Series(wholeText)

vectorizer = TfidfVectorizer()
matrix = vectorizer.fit_transform(wholeText)
messageToVectorDictionary = dict(zip(wholeText, matrix))


In [51]:
#does cosine comparison and absoluteDifference to the messages vectors and adds them into the training data

def FeatureDistinction(message1, message2):
    vector1 = messageToVectorDictionary[message1]
    vector2 = messageToVectorDictionary[message2]
    
    cosineSimilarity = cosine_similarity(vector1, vector2)[0][0]  
    #absoluteDifference = np.abs(vector1 - vector2).toarray()[0]
    
    #return np.hstack([cosineSimilarity, absoluteDifference])
    return cosineSimilarity

featureList = []
for blank, row in data.iterrows():
    featureList.append(FeatureDistinction(row['Message1'], row['Message2']))

#adds features and maps True, False to 0 and 1 
data = data.assign(MessageFeature = featureList)

data['FromSameUserNum'] = data.FromSameUser.map({
    True: 1,
    False: 0
})

data.head()

,Message1,Message2,FromSameUser,MessageFeature,FromSameUserNum
0,I made this cool @ tiz ¤ whats that @ jaws yea...,morninngg ok its a kiddy pool then laying in a...,True,0.066102,1
1,my mic iz wierd 2 ¤ / not alive WHERE ARE YOU ...,SUPER nah you dint I forget how to bold whoops...,False,0.039541,0
2,one time i got a jury duty notice in the mail ...,"dr coffeee , you missed btw if you nom someone...",True,0.161315,1
3,Text em and say something ? doubtful detective...,i like thicc but only if its kinda doughy insi...,False,0.126237,0
4,Fantastic beats by Dr Dre someone else said sa...,Fantastic beats was ok pennies ¤ all the canad...,True,0.069076,1


In [59]:
#trains the model and gives results on the effectivenes 

X = np.vstack(data["MessageFeature"])  

X_train, X_test, y_train, y_test = train_test_split(
    X,
    data.FromSameUserNum,
    test_size = 0.2, 
    random_state = 40,
    stratify = data.FromSameUserNum
)


classifier = RandomForestClassifier(n_estimators=100, random_state=40)
classifier.fit(X_train, y_train)

y_prediction = classifier.predict(X_test)
print(classification_report(y_test, y_prediction))


              precision    recall  f1-score   support

           0       0.68      0.72      0.70       104
           1       0.70      0.66      0.68       104

    accuracy                           0.69       208
   macro avg       0.69      0.69      0.69       208
weighted avg       0.69      0.69      0.69       208



In [53]:
data.FromSameUser.value_counts()

FromSameUser
True     519
False    519
Name: count, dtype: int64